In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

In [14]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

# --- ROI calculation function ---
def churn_roi_summary(churn_rate, total_customers, customer_ltv, retention_cost, recall, precision):
    churned_customers = total_customers * churn_rate
    true_positives = churned_customers * recall
    predicted_positives = true_positives / precision if precision > 0 else 0
    false_positives = max(predicted_positives - true_positives, 0)

    retained_revenue = true_positives * customer_ltv
    retention_spend = (true_positives + false_positives) * retention_cost
    net_savings = retained_revenue - retention_spend
    roi = net_savings / retention_spend if retention_spend > 0 else 0

    # Prepare horizontal DataFrame
    metrics = [
        "Churn Rate",
        "Total Customers",
        "Predicted Churned (TP+FP)",
        "True Churned (TP)",
        "False Positives",
        "Customer LTV ($)",
        "Retention Cost ($)",
        "Revenue Protected ($)",
        "Retention Spend ($)",
        "Net Savings ($)",
        "ROI"
    ]
    values = [
        f"{churn_rate:.2%}",
        f"{total_customers:,.0f}",
        f"{predicted_positives:,.0f}",
        f"{true_positives:,.0f}",
        f"{false_positives:,.0f}",
        f"${customer_ltv:,.0f}",
        f"${retention_cost:,.0f}",
        f"${retained_revenue:,.0f}",
        f"${retention_spend:,.0f}",
        f"${net_savings:,.0f}",
        f"{roi:.2f}"
    ]

    # Color coding ROI & Net Savings
    colored_values = []
    for metric, val in zip(metrics, values):
        if metric in ["Net Savings ($)", "ROI"]:
            try:
                number = float(val.replace('$','').replace(',',''))
            except:
                number = float(val)
            color = "green" if number > 0 else "red"
            val = f"<span style='color:{color}; font-weight:bold'>{val}</span>"
        colored_values.append(val)

    df_horizontal = pd.DataFrame([colored_values], columns=metrics)

    # --- Centered table with proper grid lines using CSS ---
    html_table = df_horizontal.to_html(index=False, escape=False)
    styled_html = f"""
    <div style='display:flex; justify-content:center; margin-top:20px'>
        <table style='border-collapse: collapse; border: 1px solid black;'>
            <thead>
                {html_table.split('<thead>')[1].split('</thead>')[0]}
            </thead>
            <tbody>
                {html_table.split('<tbody>')[1].split('</tbody>')[0]}
            </tbody>
        </table>
    </div>
    <style>
        table, th, td {{
            border: 1px solid black;
            padding: 8px;
            text-align: center;
        }}
        th {{
            background-color: #f0f0f0;
        }}
    </style>
    """
    display(HTML(styled_html))

# --- Title ---
title_html = HTML("<h2 style='color: #2E86C1; text-align:center;'>Customer Churn ROI Calculator</h2>")
display(title_html)

# --- Widgets ---
recall_input = widgets.FloatSlider(value=0.6, min=0.0, max=1.0, step=0.01, description='Recall', style={'description_width': '120px'}, readout_format='.0%')
precision_input = widgets.FloatSlider(value=0.7, min=0.0, max=1.0, step=0.01, description='Precision', style={'description_width': '120px'}, readout_format='.0%')
model_metrics_box = widgets.VBox([widgets.HTML("<b>Model Metrics</b>"), recall_input, precision_input])

churn_rate_input = widgets.FloatSlider(value=0.25, min=0.01, max=1.0, step=0.01, description='Churn Rate', style={'description_width': '120px'}, readout_format='.0%')
total_customers_input = widgets.IntText(value=10000, description='Total Customers', style={'description_width': '120px'})
customer_ltv_input = widgets.FloatText(value=1000, description='Customer LTV ($)', style={'description_width': '120px'})
retention_cost_input = widgets.FloatText(value=50, description='Retention Cost ($)', style={'description_width': '120px'})
financial_inputs_box = widgets.VBox([widgets.HTML("<b>Financial Inputs</b>"), churn_rate_input, total_customers_input, customer_ltv_input, retention_cost_input])

ui = widgets.HBox([model_metrics_box, financial_inputs_box], layout=widgets.Layout(justify_content='space-around'))

out = widgets.interactive_output(
    churn_roi_summary,
    {
        'churn_rate': churn_rate_input,
        'total_customers': total_customers_input,
        'customer_ltv': customer_ltv_input,
        'retention_cost': retention_cost_input,
        'recall': recall_input,
        'precision': precision_input
    }
)

display(ui)
display(widgets.HTML("<br><br>"))  # Extra space
display(out)


HTML(value='<br><br>')

Output()

In [28]:
import ipywidgets as widgets
from ipywidgets import VBox, Layout, Label
from IPython.display import display, HTML
import pandas as pd

# --- ROI calculation function ---
def churn_roi_summary(churn_rate, total_customers, customer_ltv, retention_cost, recall, precision):
    churned_customers = total_customers * churn_rate
    true_positives = churned_customers * recall
    predicted_positives = true_positives / precision if precision > 0 else 0
    false_positives = max(predicted_positives - true_positives, 0)

    retained_revenue = true_positives * customer_ltv
    retention_spend = (true_positives + false_positives) * retention_cost
    net_savings = retained_revenue - retention_spend
    roi = net_savings / retention_spend if retention_spend > 0 else 0

    # --- Prepare all metrics ---
    metrics = [
        "Churn Rate",
        "Total Customers",
        "Predicted Churned",
        "True Churned",
        "False Positives",
        "Customer LTV",
        "Retention Cost",
        "Revenue Protected",
        "Retention Spend",
        "Net Savings",
        "ROI"
    ]
    values = [
        f"{churn_rate:.2%}",
        f"{total_customers:,.0f}",
        f"{predicted_positives:,.0f}",
        f"{true_positives:,.0f}",
        f"{false_positives:,.0f}",
        f"${customer_ltv:,.0f}",
        f"${retention_cost:,.0f}",
        f"${retained_revenue:,.0f}",
        f"${retention_spend:,.0f}",
        f"${net_savings:,.0f}",
        f"{roi:.2f}"
    ]

    # --- Split into two tables ---
    # Table 1: Summary (all except Net Savings & ROI)
    summary_metrics = metrics[:-2]
    summary_values = values[:-2]
    df_summary = pd.DataFrame([summary_values], columns=summary_metrics)

    # Table 2: Business impact (Net Savings & ROI)
    impact_metrics = metrics[-2:]
    impact_values = values[-2:]
    # Apply color coding
    colored_impact_values = []
    for metric, val in zip(impact_metrics, impact_values):
        try:
            number = float(val.replace('$','').replace(',',''))
        except:
            number = float(val)
        color = "green" if number > 0 else "red"
        val = f"<span style='color:{color}; font-weight:bold'>{val}</span>"
        colored_impact_values.append(val)
    df_impact = pd.DataFrame([colored_impact_values], columns=impact_metrics)

    # --- Function to render table with borders ---
    def render_table(df):
        # html_table = df.to_html(index=False, escape=False)
        # styled_html = f"""
        # <div style='display:flex; justify-content:center; margin-top:15px'>
        #     <table style='border-collapse: collapse;'>
        #         <thead>
        #             {html_table.split('<thead>')[1].split('</thead>')[0]}
        #         </thead>
        #         <tbody>
        #             {html_table.split('<tbody>')[1].split('</tbody>')[0]}
        #         </tbody>
        #     </table>
        # </div>
        # <style>
        #     table, th, td {{
        #         border: 1px solid black;       /* grid lines for all cells */
        #         padding: 8px;
        #         text-align: center;
        #     }}
        #     th {{
        #         background-color: #f0f0f0;
        #     }}
        # </style>
        # """
        # display(HTML(styled_html))

        html_table = df.to_html(index=False, escape=False)
        styled_html = f"""
        <div style='display:flex; justify-content:center; margin-top:20px'>
            <table style='border-collapse: collapse; border: 1px solid black;'>
                <thead>
                    {html_table.split('<thead>')[1].split('</thead>')[0]}
                </thead>
                <tbody>
                    {html_table.split('<tbody>')[1].split('</tbody>')[0]}
                </tbody>
            </table>
        </div>
        <style>
            table, th, td {{
                border: 1px solid black;
                padding: 8px;
                text-align: center;
            }}
            th {{
                background-color: #f0f0f0;
            }}
        </style>
        """
        display(HTML(styled_html))

    render_table(df_impact)
    render_table(df_summary)

# --- Title ---
title_html = HTML("<h2 style='color: #2E86C1; text-align:center;'>Customer Churn ROI Calculator</h2>")
display(title_html)

# --- Widgets ---
recall_input = widgets.FloatSlider(value=0.6, min=0.0, max=1.0, step=0.01, description='Recall', style={'description_width': '120px'}, readout_format='.0%')
precision_input = widgets.FloatSlider(value=0.7, min=0.0, max=1.0, step=0.01, description='Precision', style={'description_width': '120px'}, readout_format='.0%')
model_metrics_box = widgets.VBox([widgets.HTML("<b>Model Metrics</b>"), recall_input, precision_input], layout=Layout(align_items='center'))

churn_rate_input = widgets.FloatSlider(value=0.25, min=0.01, max=1.0, step=0.01, description='Churn Rate', style={'description_width': '120px'}, readout_format='.0%')
total_customers_input = widgets.IntText(value=10000, description='Total Customers', style={'description_width': '120px'})
customer_ltv_input = widgets.FloatText(value=1000, description='Customer LTV ($)', style={'description_width': '120px'})
retention_cost_input = widgets.FloatText(value=50, description='Retention Cost ($)', style={'description_width': '120px'})
financial_inputs_box = widgets.VBox([widgets.HTML("<b>Financial Inputs</b>"), churn_rate_input, total_customers_input, customer_ltv_input, retention_cost_input],layout=Layout(align_items='center'))

ui = widgets.HBox([model_metrics_box, financial_inputs_box], layout=widgets.Layout(justify_content='space-around'))

out = widgets.interactive_output(
    churn_roi_summary,
    {
        'churn_rate': churn_rate_input,
        'total_customers': total_customers_input,
        'customer_ltv': customer_ltv_input,
        'retention_cost': retention_cost_input,
        'recall': recall_input,
        'precision': precision_input
    }
)

display(ui)
display(widgets.HTML("<br><br>"))  # Extra space
display(out)


HTML(value='<br><br>')

Output()